### Pais x Filhos: dados de Galton (1886)

  - Flavio Lichtenstein
  - Jacopo Ferretti

https://www.kaggle.com/code/jacopoferretti/parents-vs-child-heights-multilevel-linear-regr

https://www.kaggle.com/code/jacopoferretti/parents-vs-child-heights-multilevel-linear-regr/notebook

In [ ]:
import numpy as np
import pandas as pd
import os, sys

import statsmodels.api as sm
from scipy import stats

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error,r2_score,mean_squared_error
from sklearn.preprocessing import LabelEncoder,StandardScaler,MinMaxScaler
from sklearn.ensemble import RandomForestRegressor

from warnings import simplefilter
simplefilter("ignore")

# bibliotecas gráficas
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
root_data = '../data'
files = os.listdir(root_data)

# 5 primeiros
files[:5]

In [ ]:
files = [x for x in files if 'galton' in x]
files

### Este arquivo existe?

In [ ]:
fname = 'galton_families.csv'

filename = os.path.join(root_data, fname)

if not os.path.exists(filename):
    print(f"Este arquivo não existe: '{filename}'")
else:
    print(f"Este arquivo existe: '{filename}'")

### Lendo dados com panda

In [ ]:
df = pd.read_csv(filename, sep=',', header=0)
len(df)

In [ ]:
df.columns

In [ ]:
df.head(3)

In [ ]:
df.tail(3)

### Tem dados incompletos?

In [ ]:
df.isna().sum()

### Tem dados duplicados?

In [ ]:
df.duplicated().sum()

### Tipos de dados em cada coluna

In [ ]:
df.dtypes

### Estatísica descritva (um só comando)

In [ ]:
df.columns

In [ ]:
cols = ['father_height', 'mother_height', 'midparent_height', 'child_height',]
df[cols].describe()

In [ ]:
pl_p_metro = 0.0254

df['alt_pai'] = df.father_height*pl_p_metro
df['alt_mae'] = df.mother_height*pl_p_metro
df['alt_filho'] = df.child_height*pl_p_metro

df.columns

In [ ]:
cols = ['family', 'child_num', 'gender', 'alt_pai', 'alt_mae', 'alt_filho']
df2 = df[cols].copy()
df2.head(3)

In [ ]:
cols2 = ['alt_pai', 'alt_mae', 'alt_filho']
df2[cols2].describe()

In [ ]:
import matplotlib as mpl
label_size = 12
mpl.rcParams['xtick.labelsize'] = label_size 
mpl.rcParams['ytick.labelsize'] = label_size 

In [ ]:
fig = plt.figure(figsize=(12,12))
(topfig,bottomfig) = fig.subfigures(2,1);

fig.suptitle('Alturas de Pais x Filhos')


topfig.subplots_adjust(left=.1,right=.9,wspace=.4,hspace=.4);
fig, axes = plt.subplots(1,2,figsize=(12,6));

ax = axes[0]
ax.set_title("Distribuição de altura dos pais",fontsize=20)
sns.kdeplot(y=df2['alt_pai'],color='black', ax=ax)
ax.set_ylabel('altura [metros]',fontsize=15)
ax.set_xlabel('densidade',fontsize=15)
ax.set_facecolor('lemonchiffon')

med = df2['alt_pai'].median()
ax.axline((0, med), (1, med), linewidth=2, color='red')

ax = axes[1]
sns.boxplot(df2['alt_pai'],color='gold',ax=ax)
ax.set_title("Boxplot - altura Pais",fontsize=20)
ax.set_xlabel(' ')
ax.set_ylabel('altura (m)',fontsize=15)
ax.set_facecolor('lemonchiffon')

fig.set_facecolor('tan')

plt.tight_layout();

bottomfig.subplots_adjust(left=.1,right=.9,wspace=.4,hspace=.4);
fig, axes = plt.subplots(1,2,figsize=(12,6));

ax = axes[0]
ax.set_title("Distribuição de altura das mães",fontsize=20)
sns.kdeplot(data=df2, y='alt_mae',color='black', common_grid=True, ax=ax)
ax.set_ylabel('altura [metros]',fontsize=15)
ax.set_xlabel('densidade',fontsize=15)
ax.set_facecolor('lemonchiffon')


med = df2['alt_mae'].median()
ax.axline((0, med), (1, med), linewidth=2, color='red')

ax = axes[1]
sns.boxplot(df2['alt_mae'],color='gold',ax=ax)
ax.set_title("Boxplot - altura Mães",fontsize=20)
ax.set_xlabel(' ')
ax.set_ylabel('altura (m)',fontsize=15)
ax.set_facecolor('lemonchiffon')


fig.set_facecolor('tan')
plt.tight_layout();

In [ ]:
df2.head(3)

In [ ]:
df2.gender.value_counts()

### Family size

In [ ]:
vals_gender = df2.gender.value_counts()

fig,(ax1,ax2) = plt.subplots(ncols=2,figsize=(12,5))
fig.suptitle("Distribuições de crianças por gênero",fontsize=28)

ax=ax1
ax.pie(vals_gender,labels=['Filhos','Filhas'],autopct="%1.1f%%",
       shadow=True,explode=(0.03,0.03),
       colors=['gold','cadetblue'])

ax.set_title('Gênero das crianças',fontsize=20)

ax=ax2
sns.histplot(data=df2,x='alt_filho',hue='gender',
             multiple='stack', palette=['gold','cadetblue'],ax=ax)

ax.set_title('Número de crianças por família',fontsize=20)
ax.set_facecolor('lemonchiffon')
ax.set_ylabel('Contagem')

plt.tight_layout()
fig.set_facecolor('tan');

### Separando filhos e filhas

In [ ]:
df3 = df2.copy()

df3['h_filha'] = df3[df3['gender'] == 'female']['alt_filho']
df3['h_filho'] = df3[df3['gender'] == 'male']['alt_filho']

df3.head(3)

In [ ]:
fig = plt.figure(figsize=(12,12))
(topfig,bottomfig) = fig.subfigures(2,1);

fig.suptitle('Filhos x Filhas')


topfig.subplots_adjust(left=.1,right=.9,wspace=.4,hspace=.4);
fig, axes = plt.subplots(1,2,figsize=(12,6));

ax = axes[0]
ax.set_title("Distribuição de altura dos filhos",fontsize=20)
sns.kdeplot(y=df3['h_filho'],color='black', ax=ax)
ax.set_ylabel('altura [metros]',fontsize=15)
ax.set_xlabel('densidade',fontsize=15)
ax.set_facecolor('lemonchiffon')

med = df3['h_filho'].median()
ax.axline((0, med), (1, med), linewidth=2, color='red')

ax = axes[1]
sns.boxplot(df3['h_filho'],color='gold',ax=ax)
ax.set_title("Boxplot - altura dos filhos",fontsize=20)
ax.set_xlabel(' ')
ax.set_ylabel('altura (m)',fontsize=15)
ax.set_facecolor('lemonchiffon')

fig.set_facecolor('tan')
plt.tight_layout();

bottomfig.subplots_adjust(left=.1,right=.9,wspace=.4,hspace=.4);
fig, axes = plt.subplots(1,2,figsize=(12,6));

ax = axes[0]
ax.set_title("Distribuição de altura das filhas",fontsize=20)
sns.kdeplot(data=df3, y='h_filha',color='black', common_grid=True, ax=ax)
ax.set_ylabel('altura [metros]',fontsize=15)
ax.set_xlabel('densidade',fontsize=15)
ax.set_facecolor('lemonchiffon')


med = df3['h_filha'].median()
ax.axline((0, med), (1, med), linewidth=2, color='red')

ax = axes[1]
sns.boxplot(df3['h_filha'],color='gold',ax=ax)
ax.set_title("Boxplot - altura das filhas",fontsize=20)
ax.set_xlabel(' ')
ax.set_ylabel('altura (m)',fontsize=15)
ax.set_facecolor('lemonchiffon')


fig.set_facecolor('tan')
plt.tight_layout();

### Comparando as 2 gerações

In [ ]:
# !pip install joypy

In [ ]:
import joypy

In [ ]:
fig,(ax) = plt.subplots(ncols=1,figsize=(12,5))

ax = joypy.joyplot(df3,column=['alt_pai','h_filho','alt_mae','h_filha'],
                   ylim ='own',
                   title="Comaparando-se pais e filhos (em metros)",
                   figsize=(12,8),
                   legend=True,ax=ax,color=['cadetblue','gold','cadetblue','gold'])

plt.tight_layout()
plt.gca().set_facecolor('lemonchiffon')
fig.set_facecolor('tan')

### Retirar outliers

### Comparar e fazer regressões lineares

  - Pais x Mães
  - Pais x Filhos
  - Mães x Filhas

In [ ]:
df3['meia_alt_pais'] = (df3.alt_pai + df3.alt_mae) /2

In [ ]:
df3.head(3)

In [ ]:
def remove_outlier(df: pd.DataFrame, features: list) -> pd.DataFrame:

    for col in features:
        if col not in df.columns:
            print("Wrong column", col)
            return df
            
    df_out = df.copy()
    
    for column in features:     
        
        # First define the first and third quartiles
        Q1 = df_out[column].quantile(0.25)
        Q3 = df_out[column].quantile(0.75)
        # Define the inter-quartile range
        IQR = Q3 - Q1
        # ... and the lower/higher threshold values
        lowerL = Q1 - 1.5 * IQR
        higherL = Q3 + 1.5 * IQR
        
        # Impute 'left' outliers
        df_out.loc[df_out[column] < lowerL,column] = lowerL
        # Impute 'right' outliers
        df_out.loc[df_out[column] > higherL,column] = higherL

    # renumera os registros
    df_out.reset_index(inplace=True, drop=True)

    return df_out


In [ ]:
features = ['alt_pai','alt_mae','meia_alt_pais', 'alt_filho']

print(len(df3))
df4 = remove_outlier(df3,features)
print(len(df4))
df4.head(3)

### Regressão: pais x maẽs

In [ ]:
model = sm.OLS.from_formula("alt_pai ~ alt_mae",data=df4)
result = model.fit()
result.summary()

In [ ]:
fig = plt.figure(figsize=(12,5));
(topfig,bottomfig) = fig.subfigures(2,1);

topfig.subplots_adjust(left=.1,right=.9,wspace=.4,hspace=.4);
fig,axes = plt.subplots(1,2,figsize=(12,5));

ax=axes[0]
sns.regplot(x="alt_mae",y="alt_pai", data=df4, color='cadetblue', ax=ax)
ax.xaxis.set_tick_params(labelsize=15)
ax.yaxis.set_tick_params(labelsize=15)
ax.set_facecolor('lemonchiffon')
ax.set_xlabel("altura das mães (metros)",fontsize=18)
ax.set_ylabel("altura dos pais (metros)",fontsize=18)
ax.set_title("Linearidade",fontsize=22)

mae_min = df4.alt_mae.min()
mae_max = df4.alt_mae.max()
mae_med = (mae_min + mae_max)/2

pai_min = df4.alt_pai.min()
pai_max = df4.alt_pai.max()
pai_med = (pai_min + pai_max)/2

ax.axline((mae_min, pai_med), (mae_max, pai_med), linewidth=1, color='black')
ax.axline((mae_med, pai_min), (mae_med, pai_max), linewidth=1, color='black')

#--------------

ax=axes[1]
ax.plot(result.fittedvalues, result.resid_pearson, "o", alpha=0.3, color='cadetblue') 
ax.xaxis.set_tick_params(labelsize=15)
ax.yaxis.set_tick_params(labelsize=15)
ax.set_facecolor('lemonchiffon')
ax.set_xlabel("valores fitados",fontsize=18)
ax.set_ylabel('resíduos',fontsize=18)
ax.set_title("Resíduos x Valores fitados",fontsize=22)

resi_min = np.min(result.fittedvalues)
resi_max = np.max(result.fittedvalues)

ax.axline((resi_min, 0), (resi_max, 0), linewidth=1, color='black')

#--------------

fig.suptitle("Checagem da definição de Modelo de Regressão Linear",fontsize=30)
fig.set_facecolor('tan')
plt.tight_layout()

bottomfig.subplots_adjust(left=.1,right=.9,wspace=.4,hspace=.4);
fig,axes = plt.subplots(1,2,figsize=(12,5));

res = result.resid 
ax=axes[0]
sns.kdeplot(res, color='cadetblue',ax=ax)
ax.xaxis.set_tick_params(labelsize=15)
ax.yaxis.set_tick_params(labelsize=15)
ax.set_xlabel("residual value",fontsize=18)
ax.set_ylabel('count',fontsize=18)
ax.set_facecolor('lemonchiffon')
ax.set_title("Histograma of Resíduos",fontsize=22);

axes[1] = stats.probplot(result.resid,dist="norm",plot=plt)
plt.legend(['Real','Teórico'])
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.xlabel("Quartis teóricos",fontsize=18)
plt.ylabel('Valores ordenados',fontsize=18)
plt.gca().set_facecolor('lemonchiffon')
plt.title("Q-Q Plot - Resíduos",fontsize=22)

fig.set_facecolor('tan')
plt.tight_layout()

### Pais x Filhos

In [ ]:
model = sm.OLS.from_formula("alt_filho ~ alt_pai",data=df4)
result = model.fit()
result.summary()

In [ ]:
fig = plt.figure(figsize=(12,5));
(topfig,bottomfig) = fig.subfigures(2,1);

topfig.subplots_adjust(left=.1,right=.9,wspace=.4,hspace=.4);
fig,axes = plt.subplots(1,2,figsize=(12,5));

ax=axes[0]
sns.regplot(x="alt_pai",y="h_filho", data=df4, color='cadetblue', ax=ax)
ax.xaxis.set_tick_params(labelsize=15)
ax.yaxis.set_tick_params(labelsize=15)
ax.set_facecolor('lemonchiffon')
ax.set_xlabel("altura das pais (metros)",fontsize=18)
ax.set_ylabel("altura dos filhos (metros)",fontsize=18)
ax.set_title("Linearidade",fontsize=22)

pai_min = df4.alt_pai.min()
pai_max = df4.alt_pai.max()
pai_med = (pai_min + pai_max)/2

filho_min = df4.h_filho.min()
filho_max = df4.h_filho.max()
filho_med = (filho_min + filho_max)/2

ax.axline((pai_min, filho_med), (pai_max, filho_med), linewidth=1, color='black')
ax.axline((pai_med, filho_min), (pai_med, filho_max), linewidth=1, color='black')

#--------------

ax=axes[1]
ax.plot(result.fittedvalues, result.resid_pearson, "o", alpha=0.3, color='cadetblue') 
ax.xaxis.set_tick_params(labelsize=15)
ax.yaxis.set_tick_params(labelsize=15)
ax.set_facecolor('lemonchiffon')
ax.set_xlabel("valores fitados",fontsize=18)
ax.set_ylabel('resíduos',fontsize=18)
ax.set_title("Resíduos x Valores fitados",fontsize=22)

resi_min = np.min(result.fittedvalues)
resi_max = np.max(result.fittedvalues)

ax.axline((resi_min, 0), (resi_max, 0), linewidth=1, color='black')

#--------------

fig.suptitle("Checagem da definição de Modelo de Regressão Linear",fontsize=30)
fig.set_facecolor('tan')
plt.tight_layout()

bottomfig.subplots_adjust(left=.1,right=.9,wspace=.4,hspace=.4);
fig,axes = plt.subplots(1,2,figsize=(12,5));

res = result.resid 
ax=axes[0]
sns.kdeplot(res, color='cadetblue',ax=ax)
ax.xaxis.set_tick_params(labelsize=15)
ax.yaxis.set_tick_params(labelsize=15)
ax.set_xlabel("residual value",fontsize=18)
ax.set_ylabel('count',fontsize=18)
ax.set_facecolor('lemonchiffon')
ax.set_title("Histograma de Resíduos",fontsize=22);

axes[1] = stats.probplot(result.resid,dist="norm",plot=plt)
plt.legend(['Real','Teórico'])
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.xlabel("Quartis teóricos",fontsize=18)
plt.ylabel('Valores ordenados',fontsize=18)
plt.gca().set_facecolor('lemonchiffon')
plt.title("Q-Q Plot - Resíduos",fontsize=22)

fig.set_facecolor('tan')
plt.tight_layout()

### Mães x Filhas

In [ ]:
model = sm.OLS.from_formula("h_filha ~ alt_mae",data=df4)
result = model.fit()
result.summary()

In [ ]:
fig = plt.figure(figsize=(12,5));
(topfig,bottomfig) = fig.subfigures(2,1);

topfig.subplots_adjust(left=.1,right=.9,wspace=.4,hspace=.4);
fig,axes = plt.subplots(1,2,figsize=(12,5));

ax=axes[0]
sns.regplot(x="alt_mae",y="h_filha", data=df4, color='cadetblue', ax=ax)
ax.xaxis.set_tick_params(labelsize=15)
ax.yaxis.set_tick_params(labelsize=15)
ax.set_facecolor('lemonchiffon')
ax.set_xlabel("altura das mães (metros)",fontsize=18)
ax.set_ylabel("altura dos filhas (metros)",fontsize=18)
ax.set_title("Linearidade",fontsize=22)

mae_min = df4.alt_mae.min()
mae_max = df4.alt_mae.max()
mae_med = (mae_min + mae_max)/2

filha_min = df4.h_filha.min()
filha_max = df4.h_filha.max()
filha_med = (filha_min + filha_max)/2

ax.axline((mae_min, filha_med), (mae_max, filha_med), linewidth=1, color='black')
ax.axline((mae_med, filha_min), (mae_med, filha_max), linewidth=1, color='black')

#--------------

ax=axes[1]
ax.plot(result.fittedvalues, result.resid_pearson, "o", alpha=0.3, color='cadetblue') 
ax.xaxis.set_tick_params(labelsize=15)
ax.yaxis.set_tick_params(labelsize=15)
ax.set_facecolor('lemonchiffon')
ax.set_xlabel("valores fitados",fontsize=18)
ax.set_ylabel('resíduos',fontsize=18)
ax.set_title("Resíduos x Valores fitados",fontsize=22)

resi_min = np.min(result.fittedvalues)
resi_max = np.max(result.fittedvalues)

ax.axline((resi_min, 0), (resi_max, 0), linewidth=1, color='black')

#--------------

fig.suptitle("Checagem da definição de Modelo de Regressão Linear",fontsize=30)
fig.set_facecolor('tan')
plt.tight_layout()

bottomfig.subplots_adjust(left=.1,right=.9,wspace=.4,hspace=.4);
fig,axes = plt.subplots(1,2,figsize=(12,5));

res = result.resid 
ax=axes[0]
sns.kdeplot(res, color='cadetblue',ax=ax)
ax.xaxis.set_tick_params(labelsize=15)
ax.yaxis.set_tick_params(labelsize=15)
ax.set_xlabel("residual value",fontsize=18)
ax.set_ylabel('count',fontsize=18)
ax.set_facecolor('lemonchiffon')
ax.set_title("Histograma de Resíduos",fontsize=22);

axes[1] = stats.probplot(result.resid,dist="norm",plot=plt)
plt.legend(['Real','Teórico'])
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.xlabel("Quartis teóricos",fontsize=18)
plt.ylabel('Valores ordenados',fontsize=18)
plt.gca().set_facecolor('lemonchiffon')
plt.title("Q-Q Plot - Resíduos",fontsize=22)

fig.set_facecolor('tan')
plt.tight_layout()